In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
def train_val_dataset(dataset, val_split=0.1):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    return dict({
        "train" : Subset(dataset, train_idx),
        "val" : Subset(dataset, val_idx)
    })

def createDataLoader():
    mean_values = (0.5, 0.5, 0.5)
    std_values = (0.5, 0.5, 0.5)
    op_size = 32
    transform_train = transforms.Compose([
        transforms.RandomResizedCrop(op_size),
        transforms.ToTensor(),
        transforms.Normalize(mean_values,std_values ),
        ])

    transform_horizontal = transforms.Compose([
        transforms.RandomResizedCrop(op_size),                                        
        transforms.RandomHorizontalFlip(),                                       
        transforms.ToTensor(),
        ])
    transform_vertical = transforms.Compose([
        transforms.RandomResizedCrop(op_size),                                     
        transforms.RandomVerticalFlip(),                                       
        transforms.ToTensor(),
        ])
    transform_Invert= transforms.Compose([
        transforms.RandomResizedCrop(op_size),                                  
        transforms.RandomInvert(),                                       
        transforms.ToTensor(),
        ])

    dataset_url = "/content/train_local/inaturalist_12K/train"

    img_data = torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_train)
    img_data_hori= torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_horizontal)
    img_data_vert= torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_vertical)
    img_data_inve= torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_Invert)
    img_data = img_data + img_data_inve + img_data_vert + img_data_hori

    img_data = train_val_dataset(img_data)


    X_train=img_data['train']
    X_Valid=img_data['val']


    trainloader = torch.utils.data.DataLoader(X_train, batch_size=128, shuffle=True)
    validationloader = torch.utils.data.DataLoader(X_Valid, batch_size=128, shuffle=False)


    dataiter = iter(trainloader)
    images, labels = dataiter.next()
    return trainloader,validationloader


